In [95]:
using JuMP
import LinearAlgebra    
import SCS
using Hypatia

In [96]:
N, d = 2, 2

(2, 2)

In [97]:
function random_state(d)
    x = randn(ComplexF64, (d, d))
    y = x * x'
    return LinearAlgebra.Hermitian(y / LinearAlgebra.tr(y))
end

ρ = [random_state(d) for i in 1:N]
#ρ = 0.5 * [LinearAlgebra.Hermitian([[1,0] [0,1]])]
#ρ = 0.5 * [[[1,1] [1,1]],[[1,-1] [-1,1]]] # discriminate between plus and minus
#ρ = [Hermitian([1 0; 0 0]), 0.5 * Hermitian([1 1; 1 1])] # 0 and plus


ρ

2-element Vector{LinearAlgebra.Hermitian{ComplexF64, Matrix{ComplexF64}}}:
 [0.6722154032699733 + 0.0im 0.13214761957409196 - 0.12073621304135652im; 0.13214761957409196 + 0.12073621304135652im 0.3277845967300267 + 0.0im]
 [0.43731815623576686 + 0.0im -0.3820276329154072 - 0.11806911389396635im; -0.3820276329154072 + 0.11806911389396635im 0.562681843764233 + 0.0im]

In [98]:
model = Model(SCS.Optimizer)
set_silent(model)

In [99]:
E = [@variable(model, [1:d, 1:d] in HermitianPSDCone()) for i in 1:N]

2-element Vector{LinearAlgebra.Hermitian{GenericAffExpr{ComplexF64, VariableRef}, Matrix{GenericAffExpr{ComplexF64, VariableRef}}}}:
 [_[1] _[2] + _[4] im; _[2] - _[4] im _[3]]
 [_[5] _[6] + _[8] im; _[6] - _[8] im _[7]]

In [100]:
@constraint(model, sum(E) == LinearAlgebra.I)

[_[1] + _[5] - 1                  _[2] + _[6] + _[4] im + _[8] im
 _[2] + _[6] - _[4] im - _[8] im  _[3] + _[7] - 1] ∈ Zeros()

In [101]:
@objective(
    model,
    Max,
    sum(real(LinearAlgebra.tr(ρ[i] * E[i])) for i in 1:N) / N,
)

0.33610770163498666 _[1] + 0.13214761957409196 _[2] - 0.12073621304135652 _[4] + 0.16389229836501334 _[3] + 0.21865907811788343 _[5] - 0.3820276329154072 _[6] - 0.11806911389396635 _[8] + 0.2813409218821165 _[7]

In [102]:
optimize!(model)
assert_is_solved_and_feasible(model)
solution_summary(model)

* Solver : SCS

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "solved"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 7.82646e-01
  Dual objective value : 7.82649e-01

* Work counters
  Solve time (sec)   : 5.43291e-04


In [103]:
objective_value(model)

0.7826460130379089

In [104]:
0.5 + 0.25 * sum(LinearAlgebra.svdvals(ρ[1] - ρ[2]))

0.7826482002219151

In [105]:
solution = [value.(e) for e in E]

2-element Vector{Matrix{ComplexF64}}:
 [0.7077630695980093 + 0.0im 0.45478025481444745 - 0.002359009223390895im; 0.45478025481444745 + 0.002359009223390895im 0.2922369257376463 + 0.0im]
 [0.2922369207634908 + 0.0im -0.454780255344185 + 0.0023590087080407477im; -0.454780255344185 - 0.0023590087080407477im 0.7077630642834216 + 0.0im]

## Solving the dual
$$\min\text{tr[K]}\\
\text{s.c. } K \geq q_i p_i,\quad i \in 1,\ldots,N

$$ 

In [106]:
model_dual = Model(() -> Hypatia.Optimizer(verbose = true))
#model_dual = Model(SCS.Optimizer)
set_silent(model_dual)

In [ ]:
K =@variable(model_dual, [1:d, 1:d] in HermitianPSDCone())
for i in 1:N
    println(i)
    @constraint(model_dual, K >= (ρ[i]/N), HermitianPSDCone() )
    #@constraint(model_dual, K >= 0, HermitianPSDCone() )
    
end

1
2


In [108]:
model_dual

A JuMP Model
├ solver: Hypatia
├ objective_sense: FEASIBILITY_SENSE
├ num_variables: 4
├ num_constraints: 3
│ ├ Vector{VariableRef} in MOI.HermitianPositiveSemidefiniteConeTriangle: 1
│ └ Vector{AffExpr} in MOI.HermitianPositiveSemidefiniteConeTriangle: 2
└ Names registered in the model: none

In [ ]:
@objective(
    model_dual,
    Min,
    real(LinearAlgebra.tr(K))
)

_[1] + _[3]

In [110]:
optimize!(model_dual)
assert_is_solved_and_feasible(model_dual)
solution_summary(model_dual)

* Solver : Hypatia

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "Optimal"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 7.82648e-01
  Dual objective value : 7.82648e-01

* Work counters
  Solve time (sec)   : 1.58405e-03
  Barrier iterations : 9


In [111]:
println(objective_value(model))
objective_value(model_dual)

0.7826460130379089


0.7826482009212222